In [1]:
!pip install -q --upgrade google-generativeai

In [2]:
from google.colab import userdata
import getpass
import os
import google.generativeai as genai

# 실행하기 전에 열쇠창에 API키를 등록하세요.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

model = genai.GenerativeModel('gemini-pro')

In [3]:
import time
import re
from tqdm.notebook import tqdm
from google.generativeai import types as genai

# 소설 장르를 입력하세요.
prompt_genre = """{소설 장르는 현대 드라마입니다.}"""

# 작성 규칙을 입력하세요.
prompt_rule = """{당신은 한국의 매우 뛰어난 실력을 갖춘 저명한 소설 작가입니다. 이어지는 소설을 길게 작성해주세요.
스토리가 아주아주 조금씩 전개되도록 작성해주세요. 아래의 모든 조건들을 반드시 반드시 반드시!! 지켜주세요.
조건1 : 영어 사용금지.
조건2 : 중국어 사용금지.
조건3 : 숫자 사용금지.
조건4 : 스토리가 이어질 수 있는 문장으로 마무리 하시오.
조건5 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건6 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건7 : 스토리의 결말을 지어서는 안됨.
조건8 : 줄바꿈을 적절히 활용하시오.
조건9 : 대명사 사용을 자제하고, 주인공의 이름을 지속적으로 사용하시오.
조건10 : 주인공과 친구들은 절대 사망해서는 안됩니다.
조건11 : 인물들간에 대화를 자주 삽입 하시오.
조건12 : 언어는 한국어 이외에 절대 사용 금지.
조건13 : 비슷한 문장 반복 최대한 자제할것. 동일 단어 반복 최대한 자제할것.
조건14 : 특수문자 사용 최대한 자제할것.}
"""

# 등장인물을 입력하세요.
prompt_char = """{등장인물은 다음과 같습니다:
최민호, 33세, 기업가, 냉정하고 비즈니스 중심적인 성격
한지연, 28세, 예술가, 낙천적이고 자유로운 영혼
박성재, 36세, 변호사, 미소가 많은 친절한 성격
신지훈, 40세, 정치인, 적극적이고 야망 넘치는 성격
이 밖에도 추가적으로 다수의 등장인물을 등장시킬것.}
"""

# 시간-공간적 배경을 입력하세요.
prompt_back = """{시간-공간적 배경은 다음과 같습니다:
2040년, 현대 도시의 초고층 아파트에서 이 이야기는 전개됩니다.
이 도시는 현대화된 모습과 미래적인 기술로 가득 차 있으며, 사람들은 다양한 관점으로 미래에 대한 꿈을 키우고 있습니다.
하지만 이 아파트에 사는 사람들은 각자의 속마음과 비밀을 품고 살아가고 있습니다.}
"""

# 시작을 위한 지시사항을 입력하세요.
prompt_start = """{당신은 한국의 저명한 소설가입니다. 당신에게 소설을 작성해달라는 거액의 부탁이 들어와서 소설 작성 작업을 시작합니다.
당신의 모든 에너지와 집중력을 발휘해서 최고의 소설을 작성해주세요.
"""

# 마무리를 위한 지시사항을 입력하세요.
prompt_end = """{이어지는 내용으로 소설작성을 마무리 하시오. 또한 아래의 조건을 준수하시오.
조건1 : 진행되고 있던 소설을 매끄럽게 마무리 합니다. 확실하게 결말을 지으세요. 마무리 이후에는 더 이상의 아무런 문장도 생성하지 않습니다.
조건2 : 영어 사용금지.
조건3 : 중국어 사용금지.
조건4 : 숫자 사용금지.
조건5 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건6 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건7 : 줄바꿈을 적절히 활용하시오.
조건8 : 언어는 한국어 이외에 절대로 사용 금지.}"""

In [5]:
# 종종 실행이 안됩니다. 그럴경우 런타임 연결해제 및 재시작 후 다시 시도하세요.
# 생성횟수를 설정하세요.
n = 25
prompt = ""
myNovel = prompt_genre + prompt_rule + prompt_char + prompt_back
current_prompt = ""
total_tokens_accumulated = 0

def count_tokens(model, prompt):
    try:
        response = model.count_tokens(prompt)
        total_tokens = response.total_tokens
        return total_tokens
    except Exception as e:
        print(f"토큰 개수 확인 중 오류 발생: {e}")
        return None

def generate_selected_sentences(text):
    sentences = text.split('.')
    selected_sentences = ""
    intervals = [(i * (i + 1)) // 2 for i in range(1, 35)]
    reversed_intervals = intervals[::-1]
    for index in reversed_intervals:
        try:
            selected_sentences = '.'.join(sentences[-index:]).strip()
            break
        except IndexError as ie:
            print(f"IndexError 발생: {ie}")
            continue
    return selected_sentences

for i in tqdm(range(n), desc="Generating Text"):
    print()
    print(f"*** 현재 {i+1}번째 반복 중 ***")
    while True:
        if i == 0:
            current_prompt = prompt_genre + prompt_rule + prompt_char + prompt_back + prompt_start
        elif i == n - 1:
            current_prompt = prompt + prompt_end
        else:
            current_prompt = prompt + prompt_rule
        generation_config = genai.GenerationConfig(temperature=1.0)
        response = model.generate_content(current_prompt, generation_config=generation_config)
        input_tokens = count_tokens(model, current_prompt)
        print(f"사용된 입력 토큰: {input_tokens}")
        total_tokens = count_tokens(model, response.parts[0].text)
        print(f"사용된 출력 토큰: {total_tokens}")
        total_tokens_accumulated += total_tokens
        try:
            if response.candidates and response.parts:
                generated_text = response.parts[0].text
                consecutive_errors = 0
            else:
                print("모델이 적절한 후보를 생성하지 못했습니다. 재시도합니다.")
                consecutive_errors += 1
                continue
                if re.search("[^가-힣\s\n!\"'\.,?~()]", generated_text):
                    print(f"생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.")
                    continue
        except ValueError as ve:
            print(f"ValueError 발생: {ve}")
            consecutive_errors += 1
            generated_text = response.parts[0].text
        except IndexError as ie:
            print(f"IndexError 발생: {ie}")
            consecutive_errors += 1
            continue
        if consecutive_errors >= 3:
            print("연속된 예외 발생으로 인해 종료합니다.")
            break
        selected_sentences = generate_selected_sentences(myNovel)
        prompt = selected_sentences + '\n'
        myNovel += generated_text
        break
    print(f"현재까지 사용된 출력 토큰의 누적 개수: {total_tokens_accumulated}")

myNovel = myNovel[len(prompt_genre + prompt_rule + prompt_char + prompt_back):]
print(myNovel)

Generating Text:   0%|          | 0/25 [00:00<?, ?it/s]


*** 현재 1번째 반복 중 ***
사용된 입력 토큰: 687
사용된 출력 토큰: 568
현재까지 사용된 출력 토큰의 누적 개수: 568

*** 현재 2번째 반복 중 ***
사용된 입력 토큰: 981
사용된 출력 토큰: 481
현재까지 사용된 출력 토큰의 누적 개수: 1049

*** 현재 3번째 반복 중 ***
사용된 입력 토큰: 1550
사용된 출력 토큰: 422
현재까지 사용된 출력 토큰의 누적 개수: 1471

*** 현재 4번째 반복 중 ***
사용된 입력 토큰: 2031
사용된 출력 토큰: 384
현재까지 사용된 출력 토큰의 누적 개수: 1855

*** 현재 5번째 반복 중 ***
사용된 입력 토큰: 2453
사용된 출력 토큰: 400
현재까지 사용된 출력 토큰의 누적 개수: 2255

*** 현재 6번째 반복 중 ***
사용된 입력 토큰: 2837
사용된 출력 토큰: 485
현재까지 사용된 출력 토큰의 누적 개수: 2740

*** 현재 7번째 반복 중 ***
사용된 입력 토큰: 3237
사용된 출력 토큰: 601
현재까지 사용된 출력 토큰의 누적 개수: 3341

*** 현재 8번째 반복 중 ***
사용된 입력 토큰: 3722
사용된 출력 토큰: 1264
현재까지 사용된 출력 토큰의 누적 개수: 4605

*** 현재 9번째 반복 중 ***
사용된 입력 토큰: 4323
사용된 출력 토큰: 436
현재까지 사용된 출력 토큰의 누적 개수: 5041

*** 현재 10번째 반복 중 ***
사용된 입력 토큰: 5587
사용된 출력 토큰: 1544
현재까지 사용된 출력 토큰의 누적 개수: 6585

*** 현재 11번째 반복 중 ***
사용된 입력 토큰: 6023
사용된 출력 토큰: 435
현재까지 사용된 출력 토큰의 누적 개수: 7020

*** 현재 12번째 반복 중 ***
사용된 입력 토큰: 7567
사용된 출력 토큰: 559
현재까지 사용된 출력 토큰의 누적 개수: 7579

*** 현재 13번째 반복 중 ***
사용된 입력 토큰: 8002


In [ ]:
# 파일을 저장합니다. 드라이브 마운트를 해주세요.
from datetime import datetime
import os

directory_path = "/content/drive/MyDrive/novel/"

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(directory_path, f"{current_time}_novel.txt")
combined_prompt_path = os.path.join(directory_path, f"{current_time}_prompt.txt")

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

with open(file_path, "w", encoding="utf-8") as novel_file:
    novel_file.write(myNovel)

with open(combined_prompt_path, "w", encoding="utf-8") as prompt_file:
    prompt_file.write(f"Sum of Token: {total_tokens_accumulated}\n")
    prompt_file.write("\nPrompt Title:\n")
    prompt_file.write(prompt_genre + "\n")
    prompt_file.write("\nPrompt Rule:\n")
    prompt_file.write(prompt_rule + "\n")
    prompt_file.write("\nPrompt Char:\n")
    prompt_file.write(prompt_char + "\n")
    prompt_file.write("\nPrompt Back:\n")
    prompt_file.write(prompt_back + "\n")
    prompt_file.write("\nPrompt Start:\n")
    prompt_file.write(prompt_start + "\n")
    prompt_file.write("\nPrompt End:\n")
    prompt_file.write(prompt_end + "\n")

print(f"Novel 파일 저장완료: {file_path}")
print(f"Prompt 파일 저장완료: {combined_prompt_path}")